# Emergency Presentation analysis for paper

In [22]:
import pyodbc
import pandas as pd  # sure takes a long time

## SQL setup

In [28]:
conn = pyodbc.connect("DRIVER={SQL Server};SERVER=VHACDWRB03;DATABASE=ORD_Singh_202001030D")

# other stuff you can do with a connection but we won't
# cursor = conn.cursor()
# cursor.description  # gives you column names
# cursor.execute(sql)
#for i, row in enumerate(cursor):
#    if i < 20:
#        print(row)
# conn.close()

## Pandas setup

In [139]:
crcd = "Select * from dflt._ppko_crc_2019d_demographic_table"
colon_denom_df = pd.read_sql(sql=crcd, con=conn)
crcn = "Select * from dflt._ppko_crc_2019n_demographic_table"
colon_numer_df = pd.read_sql(sql=crcn, con=conn)

# Fixme - do the SQL strings and reading in data better, w/ less repetition.
# also use `with`

In [135]:
ds = colon_denom_df.shape
ds

(2375, 10)

In [136]:
colon_numer_df.shape  # Extra cols: EmergencyEventDate, EmergencyEventLocation, TypeOfEmergencyEvent

(572, 13)

In [138]:
cols_in_numer_only = ['PatientSSN', 'EmergencyEventDate', 'EmergencyEventLocation', 'TypeOfEmergencyEvent']
colon_numer_df_4col = colon_numer_df[cols_in_numer_only]
joined = pd.merge(colon_numer_df_4col, colon_denom_df, how='outer', on='PatientSSN').drop_duplicates()
# not clear why I need to drop duplicates, but I do
js = joined.shape
js
assert js[0] == ds[0]  # Number of rows in denom should not change after join.

# Analysis (colon)

## Age (flagged, not flagged, all)

In [142]:
joined[pd.notnull(joined['EmergencyEventDate'])].describe()

,PatientAge
count,572.000000
mean,71.173077
std,12.161196
min,30.000000
25%,64.000000
50%,72.000000
75%,80.000000
max,97.000000


In [143]:
joined[pd.isnull(joined['EmergencyEventDate'])].describe()

,PatientAge
count,1803.000000
mean,68.943428
std,10.445998
min,33.000000
25%,63.000000
50%,70.000000
75%,75.000000
max,98.000000


In [144]:
joined.describe()

,PatientAge
count,2375.000000
mean,69.480421
std,10.922852
min,30.000000
25%,63.000000
50%,71.000000
75%,76.000000
max,98.000000


## Categorical (flagged, not flagged, all)

In [148]:
categorical_vars = [
    #'PatientSSN',
    #'DiagnosticEventDateTime',
    #'DiagnosisEventLocation',
    #'TypeOfCancer',
    'StageOfCancer',
    #'TypeOfDiagnosisEvent',
    #'DeathDateTime',
    'PatientSex',
    #'PatientAge',
    'PatientRace'
]

for df in [joined[pd.notnull(joined['EmergencyEventDate'])],
          joined[pd.isnull(joined['EmergencyEventDate'])],
          joined]:
    for v in categorical_vars:
        display(df[v].value_counts())
    print('----\n\n\n')

IV         74
III        58
II         49
I          37
Unk/Uns    25
0          10
NA          3
Name: StageOfCancer, dtype: int64

M    548
F     24
Name: PatientSex, dtype: int64

WHITE, NOT HISPANIC    392
BLACK                  113
HISPANIC                41
OTHER                   26
Name: PatientRace, dtype: int64

----





I          210
III        173
II         135
IV         112
Unk/Uns     58
0           55
NA          10
Name: StageOfCancer, dtype: int64

...

----





I          247
III        231
IV         186
II         184
Unk/Uns     83
0           65
NA          13
Name: StageOfCancer, dtype: int64

M    2263
F     112
Name: PatientSex, dtype: int64

WHITE, NOT HISPANIC    1619
BLACK                   494
HISPANIC                148
OTHER                   114
Name: PatientRace, dtype: int64

----



